In [341]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [342]:
import requests
from tqdm import tqdm

from datetime import datetime


In [343]:
def get_energy_data():

    # get all available time stamps
    stampsurl = "https://www.smard.de/app/chart_data/410/DE/index_quarterhour.json"
    response = requests.get(stampsurl)
    #ignore first 4 years (don't need those in the baseline and speeds the code up a bit)
    timestamps = list(response.json()["timestamps"])[4*52:]

 
    col_names = ['date_time','Netzlast_Gesamt']
    energydata = pd.DataFrame(columns=col_names)
    
    # loop over all available timestamps
    for stamp in tqdm(timestamps):

        dataurl = "https://www.smard.de/app/chart_data/410/DE/410_DE_quarterhour_" + str(stamp) + ".json"
        response = requests.get(dataurl)
        rawdata = response.json()["series"]

        for i in range(len(rawdata)):

            rawdata[i][0] = datetime.fromtimestamp(int(str(rawdata[i][0])[:10])).strftime("%Y-%m-%d %H:%M:%S")

        energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])

    energydata = energydata.dropna()
    energydata["date_time"] = pd.to_datetime(energydata.date_time)
    #set date_time as index
    energydata.set_index("date_time", inplace=True)
    #resample
    energydata = energydata.resample("1h", label = "left").sum()

    return energydata

In [344]:
df = get_energy_data()

  0%|          | 0/256 [00:00<?, ?it/s]/var/folders/y3/h1kjcg6j0wj3crmk4m1ff0380000gn/T/ipykernel_86472/2942256061.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 256/256 [00:28<00:00,  8.85it/s]


In [345]:
df.head()

,Netzlast_Gesamt
date_time,
2018-12-24 00:00:00,42029.25
2018-12-24 01:00:00,39610.25
2018-12-24 02:00:00,39138.75
2018-12-24 03:00:00,39421.00
2018-12-24 04:00:00,40747.75


Rename column for convenience

In [346]:
df = df.rename(columns={"Netzlast_Gesamt": "gesamt"})

Rescale Netzlast so it fits requirements

In [347]:
df['gesamt'] = df['gesamt'] / 1000

Check dtypes and if columns contain and missing values

In [348]:
df.dtypes

gesamt    float64
dtype: object

In [349]:
df.isna().any()

gesamt    False
dtype: bool

Define weekday column

In [350]:
df["weekday"] = df.index.weekday #Monday=0, Sunday=6
#df["time"] = df.index.strftime("%H:%M")

Lead times are

In [351]:
horizons_def = [36, 40, 44, 60, 64, 68]#[24 + 12*i for i in range(5)]
horizons_def

[36, 40, 44, 60, 64, 68]

Adapt horzions so they actually fit

In [352]:
horizons = [h+1 for h in horizons_def]
horizons

[37, 41, 45, 61, 65, 69]

In [353]:
def get_date_from_horizon(last_ts, horizon):
    return last_ts + pd.DateOffset(hours=horizon)

In [354]:
LAST_IDX = -1
LAST_DATE = df.iloc[LAST_IDX].name
LAST_DATE

Timestamp('2023-11-15 21:00:00')

In [355]:
from datetime import datetime, timedelta

# Get the current date
current_date = datetime.now()

# Calculate the number of days to add or subtract to get to Thursday
# In Python's datetime module, Monday is 0 and Sunday is 6
days_until_thursday = 3 - current_date.weekday()  # 3 represents Thursday

# If it's already past Thursday, calculate for the next Thursday
if days_until_thursday < 0:
    days_until_thursday += 7

# Get the Thursday of the current week
thursday_of_current_week = current_date + timedelta(days=days_until_thursday)

# To set the time to 00:00, replace the hour, minute, second, and microsecond
thursday_of_current_week = thursday_of_current_week.replace(hour=0, minute=0, second=0, microsecond=0)

print("Thursday of the current week:", thursday_of_current_week)
LAST_DATE=thursday_of_current_week


Thursday of the current week: 2023-11-16 00:00:00


Get time and date that correspond to the lead times (starting at the last observation in our data which should be the respective thursday 0:00)  
*Attention*: if the last timestamp in the data is not thursday 0:00, you have to adjust your lead times accordingly

In [356]:
horizon_date = [get_date_from_horizon(LAST_DATE, h) for h in horizons]
horizon_date

[Timestamp('2023-11-17 13:00:00'),
 Timestamp('2023-11-17 17:00:00'),
 Timestamp('2023-11-17 21:00:00'),
 Timestamp('2023-11-18 13:00:00'),
 Timestamp('2023-11-18 17:00:00'),
 Timestamp('2023-11-18 21:00:00')]

quantile levels

In [357]:
tau = [.025, .25, .5, .75, .975]

In [358]:
#rows correspond to horizon, columns to quantile level
pred_baseline = np.zeros((6,5))

Seasonal regression

In [359]:
#seasonal regression
# Create dummy variables for months and hours
df['month'] = df.index.month
df['hour'] = df.index.hour

# Get dummies for months and hours, excluding the first month and hour to avoid multicollinearity
month_dummies = pd.get_dummies(df['month'], prefix='month', drop_first=True)
hour_dummies = pd.get_dummies(df['hour'], prefix='hour', drop_first=True)

# Join the dummies with the original DataFrame
df = df.join(month_dummies).join(hour_dummies)


In [360]:
df
print(df.dtypes)

gesamt      float64
weekday       int32
month         int32
hour          int32
month_2        bool
month_3        bool
month_4        bool
month_5        bool
month_6        bool
month_7        bool
month_8        bool
month_9        bool
month_10       bool
month_11       bool
month_12       bool
hour_1         bool
hour_2         bool
hour_3         bool
hour_4         bool
hour_5         bool
hour_6         bool
hour_7         bool
hour_8         bool
hour_9         bool
hour_10        bool
hour_11        bool
hour_12        bool
hour_13        bool
hour_14        bool
hour_15        bool
hour_16        bool
hour_17        bool
hour_18        bool
hour_19        bool
hour_20        bool
hour_21        bool
hour_22        bool
hour_23        bool
dtype: object


In [361]:
#seasonal regression
import statsmodels.api as sm

# Define the independent variables (X) and the dependent variable (y)
# Exclude the original 'month' and 'hour' columns to avoid multicollinearity
exclude_columns = ['gesamt', 'month', 'hour']
X = df[[col for col in df.columns if col not in exclude_columns]]
y = df['gesamt']
X = X.apply(pd.to_numeric)
y = y.apply(pd.to_numeric)

for column in X.columns:
    X[column] = X[column].astype(int)


# Add a constant to the model (for the intercept)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Print out the statistics
print(model.summary())
'quantile reg'

                            OLS Regression Results                            
Dep. Variable:                 gesamt   R-squared:                       0.711
Model:                            OLS   Adj. R-squared:                  0.710
Method:                 Least Squares   F-statistic:                     3009.
Date:                Wed, 15 Nov 2023   Prob (F-statistic):               0.00
Time:                        22:49:19   Log-Likelihood:            -1.3212e+05
No. Observations:               42910   AIC:                         2.643e+05
Df Residuals:                   42874   BIC:                         2.646e+05
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         57.3840      0.154    372.333      0.0

In [366]:
import pandas as pd
import statsmodels.api as sm

print("Shape of forecast_df:", forecast_df.shape)
print("Shape of X:", X.shape)



# Assuming horizon_date is a list of pandas Timestamps
horizon_dates = [pd.Timestamp('2023-11-17 13:00:00'),
                 pd.Timestamp('2023-11-17 17:00:00'),
                 pd.Timestamp('2023-11-17 21:00:00'),
                 pd.Timestamp('2023-11-18 13:00:00'),
                 pd.Timestamp('2023-11-18 17:00:00'),
                 pd.Timestamp('2023-11-18 21:00:00')]

predictions = []



# Create a new DataFrame for the forecast
forecast_df = pd.DataFrame({'date_time': horizon_dates})
print(forecast_df)
# Extract and one-hot encode month and hour from the date_time
forecast_df['month'] = forecast_df['date_time'].dt.month
forecast_df['hour'] = forecast_df['date_time'].dt.hour
forecast_df = forecast_df.join(pd.get_dummies(forecast_df['month'], prefix='month', drop_first=True))
forecast_df = forecast_df.join(pd.get_dummies(forecast_df['hour'], prefix='hour', drop_first=True))

# Drop the original month and hour columns
forecast_df.drop(['month', 'hour'], axis=1, inplace=True)

# Add constant and weekday columns
forecast_df['const'] = 1.0
forecast_df['weekday'] = forecast_df['date_time'].dt.dayofweek

# Ensure all columns in X are also in forecast_df
for col in X.columns:
    if col not in forecast_df.columns:
        forecast_df[col] = 0

# Reorder columns to match X
forecast_df = forecast_df[X.columns]

# Now forecast_df is ready for prediction
# Drop the first column if it's redundant
#forecast_df.drop(forecast_df.columns[0], axis=1, inplace=True)

print(forecast_df)



Shape of forecast_df: (6, 36)
Shape of X: (42910, 36)
            date_time
0 2023-11-17 13:00:00
1 2023-11-17 17:00:00
2 2023-11-17 21:00:00
3 2023-11-18 13:00:00
4 2023-11-18 17:00:00
5 2023-11-18 21:00:00
   const  weekday  month_2  month_3  month_4  month_5  month_6  month_7  \
0    1.0        4        0        0        0        0        0        0   
1    1.0        4        0        0        0        0        0        0   
2    1.0        4        0        0        0        0        0        0   
3    1.0        5        0        0        0        0        0        0   
4    1.0        5        0        0        0        0        0        0   
5    1.0        5        0        0        0        0        0        0   

   month_8  month_9  ...  hour_14  hour_15  hour_16  hour_17  hour_18  \
0        0        0  ...        0        0        0    False        0   
1        0        0  ...        0        0        0     True        0   
2        0        0  ...        0        0     

In [369]:
# Assuming horizon_dates is a list of pandas Timestamps
horizon_dates = [pd.Timestamp('2023-11-17 13:00:00'),
                 pd.Timestamp('2023-11-17 17:00:00'),
                 pd.Timestamp('2023-11-17 21:00:00'),
                 pd.Timestamp('2023-11-18 13:00:00'),
                 pd.Timestamp('2023-11-18 17:00:00'),
                 pd.Timestamp('2023-11-18 21:00:00')]

# Create a new DataFrame for the forecast
forecast_df = pd.DataFrame({'date_time': horizon_dates})
# Extract and one-hot encode month and hour from the date_time
forecast_df['month'] = forecast_df['date_time'].dt.month
forecast_df['hour'] = forecast_df['date_time'].dt.hour
forecast_df = forecast_df.join(pd.get_dummies(forecast_df['month'], prefix='month', drop_first=True))
forecast_df = forecast_df.join(pd.get_dummies(forecast_df['hour'], prefix='hour', drop_first=True))

# Drop the original month and hour columns
forecast_df.drop(['month', 'hour'], axis=1, inplace=True)

# Add constant and weekday columns
forecast_df['const'] = 1.0
forecast_df['weekday'] = forecast_df['date_time'].dt.dayofweek

# Ensure all columns in X are also in forecast_df
for col in X.columns:
    if col not in forecast_df.columns and col != 'date_time':
        forecast_df[col] = 0

# Reorder columns to match X (excluding date_time)
forecast_columns = [col for col in X.columns if col != 'date_time']
forecast_df = forecast_df[['date_time'] + forecast_columns]

# Now forecast_df is ready for prediction
# Drop the date_time column for prediction
forecast_df_for_prediction = forecast_df.drop('date_time', axis=1)

# Make predictions
predictions = model.predict(forecast_df_for_prediction)

# Combine predictions with date_times
forecast_df['prediction'] = predictions
print(forecast_df[['date_time', 'prediction']])
forecast_df


            date_time prediction
0 2023-11-17 13:00:00  50.292641
1 2023-11-17 17:00:00  63.819479
2 2023-11-17 21:00:00  58.879266
3 2023-11-18 13:00:00  48.519804
4 2023-11-18 17:00:00  62.046642
5 2023-11-18 21:00:00  57.106429


,date_time,const,weekday,month_2,month_3,month_4,month_5,month_6,month_7,month_8,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,prediction
0,2023-11-17 13:00:00,1.0,4,0,0,0,0,0,0,0,...,0,0,False,0,0,0,False,0,0,50.292641
1,2023-11-17 17:00:00,1.0,4,0,0,0,0,0,0,0,...,0,0,True,0,0,0,False,0,0,63.819479
2,2023-11-17 21:00:00,1.0,4,0,0,0,0,0,0,0,...,0,0,False,0,0,0,True,0,0,58.879266
3,2023-11-18 13:00:00,1.0,5,0,0,0,0,0,0,0,...,0,0,False,0,0,0,False,0,0,48.519804
4,2023-11-18 17:00:00,1.0,5,0,0,0,0,0,0,0,...,0,0,True,0,0,0,False,0,0,62.046642
5,2023-11-18 21:00:00,1.0,5,0,0,0,0,0,0,0,...,0,0,False,0,0,0,True,0,0,57.106429


In [367]:
X.head()

,const,weekday,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
date_time,,,,,,,,,,,,,,,,,,,,,
2018-12-24 00:00:00,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-12-24 01:00:00,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-12-24 02:00:00,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-12-24 03:00:00,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-12-24 04:00:00,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
predictions

In [372]:
#baseline
last_t = 100

for i,d in enumerate(horizon_date):
    
    weekday = d.weekday()
    hour = d.hour
    
    df_tmp = df.iloc[:LAST_IDX]
    
    cond = (df_tmp.weekday == weekday) & (df_tmp.index.time == d.time())
    
    pred_baseline[i,:] = np.quantile(df_tmp[cond].iloc[-last_t:]["gesamt"], q=tau)

In [373]:
pred_baseline

array([[51.66358125, 59.2420625 , 62.029375  , 65.577625  , 73.36360625],
       [51.09976875, 55.669625  , 59.1605    , 64.0443125 , 72.68700625],
       [47.7235    , 50.3398125 , 52.98975   , 56.98025   , 63.51105625],
       [45.99123125, 49.9445625 , 51.937625  , 55.6895    , 62.02273125],
       [44.6488625 , 47.9235    , 50.627875  , 55.8206875 , 64.4247625 ],
       [42.2491875 , 44.67875   , 47.618625  , 50.817125  , 57.1165625 ]])

Visually check if quantiles make sense

In [ ]:
x = horizons
_ = plt.plot(x,pred_baseline, ls="", marker="o", c="black")
_ = plt.xticks(x, x)
_ = plt.plot((x,x),(pred_baseline[:,0], pred_baseline[:,-1]),c='black')

In [ ]:
from datetime import datetime, date, timedelta
date_str = datetime.today().strftime('%Y%m%d')

In [ ]:
date_str = date.today() #- timedelta(days=1)
date_str = date_str
date_str

In [ ]:
df_sub = pd.DataFrame({
    "forecast_date": date_str,
    "target": "energy",
    "horizon": [str(h) + " hour" for h in horizons_def],
    "q0.025": pred_baseline[:,0],
    "q0.25": pred_baseline[:,1],
    "q0.5": pred_baseline[:,2],
    "q0.75": pred_baseline[:,3],
    "q0.975": pred_baseline[:,4]})
df_sub

In [ ]:
#need to change this
PATH = "../forecasts/"
date_str = date_str.strftime('%Y%m%d')

df_sub.to_csv(PATH+date_str+"_power_benchmark.csv", index=False)